## Задание 2
### Петрова Мария

## 3t^2 - 5t^2 - 11t - 3t^2 + 5t + 11

In [1]:
class Operation():
    
    def __init__(self, input_nodes = []):
        
        self.input_nodes = input_nodes # The list of input nodes
        self.output_nodes = [] # List of nodes consuming this node's output
        
        for node in input_nodes:
            node.output_nodes.append(self)
        
        _default_graph.operations.append(self)
  
    def compute(self):
        pass

In [2]:
class add(Operation):
    
    def __init__(self, x, y):
         
        super().__init__([x, y])

    def compute(self, x_var, y_var):
         
        self.inputs = [x_var, y_var]
        return x_var + y_var
    

In [3]:
class multiply(Operation):
     
    def __init__(self, a, b):
        
        super().__init__([a, b])
    
    def compute(self, a_var, b_var):
         
        self.inputs = [a_var, b_var]
        return a_var * b_var

In [24]:
class raise_power(Operation):
     
    def __init__(self, a, b):
        
        super().__init__([a, b])
    
    def compute(self, a_var, b_var):
         
        self.inputs = [a_var, b_var]
        return a_var ** b_var

In [5]:
class subtract(Operation):
    
    def __init__(self, x, y):
         
        super().__init__([x, y])

    def compute(self, x_var, y_var):
         
        self.inputs = [x_var, y_var]
        return x_var - y_var

In [6]:
class Placeholder():
    
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [7]:
class Variable():
    
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
         
        _default_graph.variables.append(self)


In [8]:
class Graph():  
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):

        global _default_graph
        _default_graph = self

### A basic graph z = A*t^D - B*t^D - C*t - A*t^D + B*t + C

In [10]:
g = Graph()

In [11]:
g.set_as_default()

In [12]:
A = Variable(3)
B = Variable(5)
C = Variable(11)
D = Variable(2)

In [13]:
t = Placeholder()

In [25]:
y = raise_power(t,D)
y1 = multiply(A,y)
y2 = multiply(B,y)
x1 = multiply(C,t)
x2 = multiply(B,t)

y3 = subtract(y1,y2)
x3 = subtract(y3,x1)
y4 = subtract(x3,y1)
x4 = add(y4,x2)
z =  add(x4,C)

### Session

In [15]:
import numpy as np

###### Traversing Operation Nodes

In [16]:
def traverse_postorder(operation):
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [26]:
class Session:
    
    def run(self, operation, feed_dict = {}):
        """ 
          operation: The operation to compute
          feed_dict: Dictionary mapping placeholders to input values (the data)  
        """
        
        # Puts nodes in correct order
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:

            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else: # Operation
                
                node.inputs = [input_node.output for input_node in node.input_nodes]

                 
                node.output = node.compute(*node.inputs)
                
            # Convert lists to numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        # Return the requested node value
        return operation.output
    
   

In [27]:
sess = Session()

In [28]:
result = sess.run(operation=z,feed_dict={t:10})

In [29]:
result

-549

In [31]:
3*10**2 - 5*10**2 - 11*10 - 3*10**2 + 5*10 + 11

-549

### Yay